In [1]:
from pathlib import Path

from statsmodels.stats.multitest import multipletests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"

n_clusters = {"SVR": {"ABCD": 4, "HCP-D": 2, "HCP": 3}, "KRR": {"ABCD": 4, "HCP-D": 2, "HCP": 4}}
clusters = {
    "ABCD": ["Verbal memory", "Cognition", "CBCL", "Prodromal psychosis"],
    "HCP-D": ["Cognition", "Emotion recognition"],
    "HCP": ["Social cognition", "Negative / positive feelings", "Emotion recognition", "Positive affect"]}
covars = [
    "Age", "Sex/gender", "Ethnicity/race", "Education", "Family income", "Head motion", "Euler characteristic", "Intracranial volume"]
covar_names = ["age", "sex", "ethnicity", "education", "income", "FD", "Euler characteristic", "ICV"]
covar_fnames = ["age", "sex", "ethnicity", "educ", "income", "FD", "Euler", "ICV"]

In [2]:
method = "SVR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 0.02, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}

# GLM results
p_vals = []
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_r2_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res_r2 = pd.read_csv(res_r2_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            lrt_p_file = Path(indir_curr, f"{method}_err{cluster+1}_importance_{covar_fname}.csv")
            lrt_p = pd.read_csv(lrt_p_file, header=None).squeeze()
            p_vals.append(lrt_p)

            r2_full = res_r2["pseudo_r_squared"].loc[res_r2["model"] == "Full model"].values[0]
            r2_reduced = res_r2["pseudo_r_squared"].loc[res_r2["model"] == f"Model without {covar_name}"].values[0]
            r2_partial = (r2_full - r2_reduced) / (1 - r2_reduced)
            lines["color"].append(r2_partial)

# Multiple comparison correction
h = multipletests(p_vals, method="fdr_bh")
for i, reject in enumerate(h[0]):
    if not reject:
        lines["color"][i] = "white"

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "SVR_glm_partial_r2.png"))

In [3]:
method = "KRR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 0.02, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}

# GLM results
p_vals = []
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_r2_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res_r2 = pd.read_csv(res_r2_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            lrt_p_file = Path(indir_curr, f"{method}_err{cluster+1}_importance_{covar_fname}.csv")
            lrt_p = pd.read_csv(lrt_p_file, header=None).squeeze()
            p_vals.append(lrt_p)

            r2_full = res_r2["pseudo_r_squared"].loc[res_r2["model"] == "Full model"].values[0]
            r2_reduced = res_r2["pseudo_r_squared"].loc[res_r2["model"] == f"Model without {covar_name}"].values[0]
            r2_partial = (r2_full - r2_reduced) / (1 - r2_reduced)
            lines["color"].append(r2_partial)

# Multiple comparison correction
h = multipletests(p_vals, method="fdr_bh")
for i, reject in enumerate(h[0]):
    if not reject:
        lines["color"][i] = "white"

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "KRR_glm_partial_r2.png"))

In [4]:
method = "SVR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 10, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}

# GLM results
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res = pd.read_csv(res_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            # AIC diff: 0-2 (no diff), 2-7 (evidence for full model), 7-10 (strong evidence), >10 (very strong evidence)
            aic_full = res["AIC"].loc[res["model"] == "Full model"].values[0]
            aic_reduced = res["AIC"].loc[res["model"] == f"Model without {covar_name}"].values[0]
            aic_diff = aic_reduced - aic_full
            if aic_diff <= 0:
                lines["color"].append("white")
            else:
                lines["color"].append(aic_diff)

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "SVR_glm_AIC.png"))

In [5]:
method = "KRR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 10, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}

# GLM results
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res = pd.read_csv(res_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            # AIC diff: 0-2 (no diff), 2-7 (evidence for full model), 7-10 (strong evidence), >10 (very strong evidence)
            aic_full = res["AIC"].loc[res["model"] == "Full model"].values[0]
            aic_reduced = res["AIC"].loc[res["model"] == f"Model without {covar_name}"].values[0]
            aic_diff = aic_reduced - aic_full
            if aic_diff <= 0:
                lines["color"].append("white")
            else:
                lines["color"].append(aic_diff)

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "KRR_glm_AIC.png"))

In [6]:
method = "SVR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 10, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}

# GLM results
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res = pd.read_csv(res_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            # BIC diff: 0-2 (weak evidence), 2-7 (evidence for full model), 7-10 (strong evidence), >10 (very strong evidence)
            bic_full = res["BIC"].loc[res["model"] == "Full model"].values[0]
            bic_reduced = res["BIC"].loc[res["model"] == f"Model without {covar_name}"].values[0]
            bic_diff = bic_reduced - bic_full
            if bic_diff <= 0:
                lines["color"].append("white")
            else:
                lines["color"].append(bic_diff)

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "SVR_glm_BIC.png"))

In [7]:
method = "KRR"

dimensions = [{"label": "Behavioral cluster", "values": []}, {"label": "Individual characteristics", "values": []}]
lines = {
    "color": [], "colorscale": px.colors.sequential.OrRd, "shape": "hspline", "cmin": 0, "cmax": 10, "showscale": True,
    "colorbar": {"nticks": 3, "tickmode": "auto", "thickness": 10, "xpad": 120, "tickfont": {"size": 16}}}
# GLM results
for dataset, cluster_names in clusters.items():
    indir_curr = Path("results", dataset, "multivariate_association")
    n_cluster = n_clusters[method][dataset]
    for cluster in range(n_cluster):
        res_file = Path(indir_curr, f"{method}_err{cluster+1}_goodness.csv")
        res = pd.read_csv(res_file)
        for covar, covar_name, covar_fname in zip(covars, covar_names, covar_fnames):
            dimensions[0]["values"].append(f"{dataset} {cluster_names[cluster]}")
            dimensions[1]["values"].append(covar)
            
            # BIC diff: 0-2 (weak evidence), 2-7 (evidence for full model), 7-10 (strong evidence), >10 (very strong evidence)
            bic_full = res["BIC"].loc[res["model"] == "Full model"].values[0]
            bic_reduced = res["BIC"].loc[res["model"] == f"Model without {covar_name}"].values[0]
            bic_diff = bic_reduced - bic_full
            if bic_diff <= 0:
                lines["color"].append("white")
            else:
                lines["color"].append(bic_diff)

f = go.Figure(go.Parcats(dimensions=dimensions, line=lines, labelfont={"size": 16}, tickfont={"size": 16}))
f.update_layout(width=1000, height=800, margin={"l": 250, "r": 150})
f.show()
f.write_image(Path("plots", "KRR_glm_BIC.png"))